In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough # Core LCEL components

# Load environment variables from a .env file. 🌍
# This ensures your OpenAI API key is loaded securely from your environment.
load_dotenv()











True

In [2]:
# Initialize the ChatOpenAI language model. 🤖
# This model will be used for both generating the joke and explaining it.
model = ChatOpenAI()

# Initialize a StrOutputParser. 📄
# This parser extracts the raw string content from the LLM's response.
parser = StrOutputParser()

In [3]:
# Define the first prompt template. 📝
# This template takes a 'topic' as input and instructs the LLM to generate a joke about it.
prompt1 = PromptTemplate(
    template='Write a dad-joke about {topic} in one sentence',
    input_variables=['topic']
)

In [4]:
# Define the second prompt template. 📝
# This template takes 'text' (which will be the joke generated in the previous step)
# and instructs the LLM to explain that joke.
prompt2 = PromptTemplate(
    template='Explain the following joke - {text} in 2 sentences',
    input_variables=['text']
)

In [5]:
# Define the first sub-chain: `joke_gen_chain`. ⛓️
# This is a sequential chain that:
# 1. Takes the initial `topic` input.
# 2. Formats it using `prompt1` to create the joke prompt.
# 3. Sends the prompt to the `model` to generate the joke.
# 4. Uses the `parser` to extract the raw joke string.
# The output of this chain will be the joke text (e.g., "Why did the scarecrow win an award? Because he was outstanding in his field!").
joke_gen_chain = RunnableSequence(prompt1, model, parser)

In [6]:
# Define the parallel processing part: `parallel_chain`. 👯
# This uses `RunnableParallel` to run multiple operations concurrently.
# The input to `parallel_chain` will be the *output of `joke_gen_chain`* (i.e., the joke text).
# It creates a dictionary output with two keys: 'joke' and 'explanation'.
parallel_chain = RunnableParallel({
    # 'joke' key:
    # `RunnablePassthrough()` takes the input it receives (which is the joke text from `joke_gen_chain`)
    # and simply passes it through unchanged. So, the 'joke' key will contain the original joke text.
    'joke': RunnablePassthrough(),

    # 'explanation' key:
    # This is another sequential chain that will:
    # 1. Take the input (the joke text from `joke_gen_chain`).
    # 2. Format it using `prompt2` (which asks to explain the joke).
    # 3. Sends the explanation prompt to the `model`.
    # 4. Uses the `parser` to extract the raw explanation string.
    'explanation': RunnableSequence(prompt2, model, parser)
})

In [7]:
# Define the final overall chain: `final_chain`. 🔗
# This chain combines `joke_gen_chain` and `parallel_chain` sequentially.
# 1. `joke_gen_chain` runs first, generating the joke text.
# 2. The *output* of `joke_gen_chain` (the joke text) becomes the *input* to `parallel_chain`.
# 3. `parallel_chain` then executes its two branches concurrently using this joke text:
#    - One branch just passes the joke through.
#    - The other branch generates an explanation for the joke.
# 4. The final output of `final_chain` will be a dictionary like:
#    `{'joke': '...', 'explanation': '...'}`
final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

# Invoke the entire `final_chain` with the initial topic. 🚀
# The input `{'topic':'cricket'}` is fed to `joke_gen_chain` to start the process.
print(final_chain.invoke({'topic':'AI'}))

{'joke': 'Why did the AI go to therapy? It had too many mental loops!', 'explanation': "The joke plays on the idea that AI processes information through loops, but in this case, the AI's excessive mental loops led to the need for therapy. Essentially, the AI's repetitive thoughts or decision-making processes were causing it distress."}
